In [39]:
import pandas as pd
import csv
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Lambda, LSTM, Dense, Dropout, Input
import os
import re
import pickle
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split
import numpy as np

## Data Preprocessing

In [18]:
print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [21]:
fin_fil_df = pd.read_pickle('fin.pkl')

In [7]:
directory_path = 'data/music4all_subset/lyrics'
file_prefixes = fin_fil_df['id'].tolist()
lyrics_d = {}
lyrics_l = []
for prefix in file_prefixes:
    file_pattern = prefix + '.txt'
    file_path = os.path.join(directory_path, file_pattern)
    if os.path.exists(file_path):
        with open(file_path, 'r', encoding="utf8") as file:
            content = file.read()
            lyrics_d[prefix] = content
            lyrics_l.append(content)
            # print(f"Content of {file_pattern}:\n{content}")

In [9]:
with open('lyrics_l.pkl', 'wb') as f:
    pickle.dump(lyrics_l, f)

In [4]:
def preprocess_lyrics(lyli):
    def prep_single(ly):
        text = re.sub(r'[^\w\s]', '', ly).lower()
        text = text.replace('\n', ' ')
        text = re.sub(' +', ' ', text)
        return text
    new = [prep_single(l) for l in lyli]
    return new

In [5]:
with open('lyrics_l.pkl', 'rb') as f:
    lyrics_l = pickle.load(f)

In [6]:
prep_lyrics = preprocess_lyrics(lyrics_l)
#prep_lyrics = prep_lyrics[:5000]

In [7]:
elmo = hub.load('https://tfhub.dev/google/elmo/2').signatures['default']

In [8]:
with tf.device('/GPU:0'):
    gpu_lyrics = tf.constant(prep_lyrics)
    embeddings = elmo(gpu_lyrics)['elmo']

ResourceExhaustedError: Graph execution error:

Detected at node 'bilm/embedding_lookup' defined at (most recent call last):
    File "C:\Users\Austin\anaconda3\lib\runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Austin\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\users\austin\downloads\multimodalgenreclassification\env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\users\austin\downloads\multimodalgenreclassification\env\lib\site-packages\traitlets\config\application.py", line 1053, in launch_instance
      app.start()
    File "c:\users\austin\downloads\multimodalgenreclassification\env\lib\site-packages\ipykernel\kernelapp.py", line 737, in start
      self.io_loop.start()
    File "c:\users\austin\downloads\multimodalgenreclassification\env\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      se

In [24]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

labels = fin_fil_df['genre']

genre_labels = encoder.fit_transform(labels)

print(encoder.classes_)

X_train, X_test, y_train, y_test = train_test_split(lyrics_l, genre_labels, test_size=0.2)

['alternative rock' 'ambient' 'classic rock' 'electronic' 'folk'
 'indie rock' 'metal' 'pop' 'rap' 'singer-songwriter' 'soul']


In [25]:
# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_tokenized_sentences = tokenizer(X_train, padding=True, truncation=True, return_tensors="tf")
test_tokenized_sentences = tokenizer(X_test, padding=True, truncation=True, return_tensors="tf")

In [27]:
print(train_tokenized_sentences['input_ids'].shape)

(35888, 512)


In [40]:
train_tokenized_sentences = train_tokenized_sentences['input_ids']
test_tokenized_sentences = test_tokenized_sentences['input_ids']



TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got 'input_ids'

In [55]:
trainX = np.reshape(train_tokenized_sentences, (train_tokenized_sentences.shape[0], 1, train_tokenized_sentences.shape[1]))
testX = np.reshape(test_tokenized_sentences, (test_tokenized_sentences.shape[0], 1, test_tokenized_sentences.shape[1]))

In [57]:
train_tokenized_sentences

<tf.Tensor: shape=(35888, 512), dtype=int32, numpy=
array([[ 101, 2205, 2172, ...,    0,    0,    0],
       [ 101, 1045, 2156, ...,    0,    0,    0],
       [ 101, 2821, 1010, ..., 1007, 2292,  102],
       ...,
       [ 101, 2651, 1045, ...,    0,    0,    0],
       [ 101, 2339, 2003, ..., 2008, 1045,  102],
       [ 101, 1045, 1005, ...,    0,    0,    0]])>

In [68]:
max_sequence_length = tokenized_sentences['input_ids'].shape[1]  # Get the maximum sequence length
num_classes = 11  # Replace with the number of classes

model = tf.keras.Sequential([
    Dense(2048, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.5),
    Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.5),
    Dense(512, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.5),
    Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.5),
    Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(.5),
    Dense(num_classes, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [70]:
model.fit(train_tokenized_sentences, y_train, validation_data=(test_tokenized_sentences, y_test), epochs=20, batch_size=32)

Epoch 1/20
1122/1122 [==============================] - 13s 12ms/step - loss: 2.0147 - accuracy: 0.3151 - val_loss: 2.0313 - val_accuracy: 0.3177
Epoch 2/20
1122/1122 [==============================] - 14s 12ms/step - loss: 2.0097 - accuracy: 0.3170 - val_loss: 2.0237 - val_accuracy: 0.3148
Epoch 3/20
1122/1122 [==============================] - 13s 12ms/step - loss: 2.0111 - accuracy: 0.3150 - val_loss: 2.0401 - val_accuracy: 0.3172
Epoch 4/20
1122/1122 [==============================] - 13s 12ms/step - loss: 2.0056 - accuracy: 0.3174 - val_loss: 2.0309 - val_accuracy: 0.3116
Epoch 5/20
1122/1122 [==============================] - 13s 12ms/step - loss: 1.9989 - accuracy: 0.3206 - val_loss: 2.0326 - val_accuracy: 0.3136
Epoch 6/20
1122/1122 [==============================] - 13s 12ms/step - loss: 1.9974 - accuracy: 0.3193 - val_loss: 2.0306 - val_accuracy: 0.3132
Epoch 7/20
1122/1122 [==============================] - 15s 13ms/step - loss: 1.9962 - accuracy: 0.3207 - val_loss: 2.0336 -

In [ ]:
type(train_tokenized_sentences['input_ids'])